# **Title Name :  도형인식서비스**

<p style="font-weight:bolder; font-size : 21px">
    Step : [Custom] YOLOv8 Modeling Transfer Learning[step:0]
<p>
<p style="font-weight:bolder; font-size : 21px">
   RegDate : 2024.01.08

------------------------------------------------------------

In [ ]:
''' 
Notice : 
   일단 최종배포파일을 합쳐서 만든다면, 개인적으로는 여기쪽에 합쳐질 메인파일을  위치시키면 어떨까 싶습니다.
   아직 팀 최종 배포파일이 어떻게 될지 모르고 각자 실행환경이랑 코드가 다를것으로 예상이 되서,
   우선 환경설정 부분만 일단 옮겨놨습니다. 
   추후 어떻게 구성할지는 이야기 나눠보면 좋을것 같습니다.
'''

In [2]:
# !nvidia-smi
# !pip install ultralytics

# 1. 환경설정
-------------------------------------------------------

In [ ]:
#===============================================================================
# ▶ 모듈 불러오기
#===============================================================================

# 시스템
import os
import sys
import random
from glob import glob
from time import time

# 데이터분석
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#이미지작업
from PIL import Image
import cv2

# 파이토치
from torch import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader , Dataset ,ConcatDataset, random_split
from torchsummary import summary
from torch.autograd import Variable

# CV용 토치비전
import torchvision
import torchvision.models as models
from torchvision.transforms import transforms

# YOLO
from ultralytics import YOLO

# 사이킷런
import sklearn
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GroupKFold, train_test_split
from sklearn.metrics import average_precision_score

# 트랜스포머
from transformers import get_linear_schedule_with_warmup


# 유틸
import gc
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# 기타추가옵션
import copy
import zipfile

# 시각화테마
sns.set_style(style='white')
plt.style.use('dark_background')

# 작업환경
IS_GOOGLE = True if 'google.colab'                 in sys.modules   else False
IS_KAGGLE = True if 'KAGGLE_KERNEL_RUN_TYPE'       in os.environ    else False
IS_LOCAL  = True if  not (IS_GOOGLE or IS_KAGGLE)                   else False

In [ ]:
#===============================================================================
# ▶ 시드설정
#===============================================================================

# 시드설정
SEED = 2023
def set_seed(SEED):
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)

# deterministic 설정
deterministic = True
if deterministic:
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark     = False

set_seed(SEED)

In [ ]:
#===============================================================================
# ▶ 버전체크
#===============================================================================
print(f'numpy_Ver   :  {np.__version__}버전')
print('-'*50)
print(f'pandas_Ver  :  {pd.__version__}버전')
print('-'*50)
print(f'seaborn_Ver :  {sns.__version__}버전')
print('-'*50)
print(f'torch_Ver   :  {torch.__version__}버전')

In [ ]:
#===============================================================================
# ▶ 디바이스설정
#===============================================================================
device      = 'cuda' if torch.cuda.is_available() else 'cpu'
str_device  = 'GPU' if torch.cuda.is_available() else 'CPU'
print(f'현재사용 디바이스 : {str_device}')